In [21]:
from dataclasses import dataclass

from click import prompt


@dataclass(frozen=True)
class Constants:
    TAX_PATH = "./tax.docx"
    TAX_WITH_TABLE_PATH = "./tax_with_table.docx" # CHATGPT 는 테이블을 인식하지 못한다.
    TAX_WITH_MARKDOWN_PATH = "./tax_with_markdown.docx" # 제일 적합
    CHROMA_DB_PATH = "./chroma"
    CHROMA_COLLECTION_NAME = "chroma-tax"
    MODEL_CHATGPT = "gpt-4o-mini"
    MODEL_EMBEDDING = "text-embedding-3-large"


In [22]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 문서를 쪼갠다.
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, # 내부 문서 객체 하나가 가질 수 있는 토큰 수
    chunk_overlap=200 # 위/아래 문맥을 겹치게 하여 문서 객체 사이의 겹치는 토큰의 정도 (유사도를 위해)
)

loader = Docx2txtLoader(Constants.TAX_WITH_MARKDOWN_PATH)
document = loader.load_and_split(text_splitter=text_splitter)
len(document) # 187개

225

In [23]:
# 임베딩을 한다.
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model=Constants.MODEL_EMBEDDING)

In [24]:
import os
import chromadb
from langchain_chroma import Chroma

exist_db = False
if os.path.isdir(Constants.CHROMA_DB_PATH):
    exist_db = True

if exist_db:
    database = Chroma(
        collection_name=Constants.CHROMA_COLLECTION_NAME,
        persist_directory=Constants.CHROMA_DB_PATH,
        embedding_function=embedding
    )
else:
    chromadb.api.client.SharedSystemClient.clear_system_cache()
    database = Chroma.from_documents(
        documents=document,
        embedding=embedding,
        collection_name=Constants.CHROMA_COLLECTION_NAME, # RDB로 치면 테이블 이름
        persist_directory=Constants.CHROMA_DB_PATH # 데이터 저장 위치
    )

In [30]:
# 소득세 자료에는 "거주자" 로 표현되어 있어서 관련이 없어 보인다.
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"
# 질문을 바꾸면 제55조가 맨 위로 나오게 되어 관련도가 높음을 알 수 있다.
# 벡터 스토어에 저장을 제대로 하는 것도 중요하지만, 질문을 제대로 하는 것도 중요하다.
#query = "연봉 5천만원인 거주자의 소득세는 얼마인가요?"

# 임베딩을 활용하여 알아서 유사도 검색 후 자료를 가져온다.
# 기본적으로 4개를 가져오는데 k 값을 바꾸면 된다.
# retrieved_docs = database.similarity_search(query, k=4)

# database 상관없이 가져오려면 아래처럼 하면 된다.
retriever = database.as_retriever(search_kwargs={'k': 4})
retrieved_docs = retriever.invoke(query)

retrieved_docs

[Document(id='184178b1-4e29-4ac0-aaa1-0b728099c8ba', metadata={'source': './tax_with_markdown.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n| 종합소득 과세표준          | 세율                                         |\n\n|-------------------|--------------------------------------------|\n\n| 1,400만원 이하     | 과세표준의 6퍼센트                             |\n\n| 1,400만원 초과     5,000만원 이하     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)  |\n\n| 5,000만원 초과   8,800만원 이하     | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) |\n\n| 8,800만원 초과 1억5천만원 이하    | 3,706만원 + (8,800만원을 초과하는 금액의 35퍼센트)|\n\n| 1억5천만원 초과 3억원 이하         | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)|\n\n| 3억원 초과    5억원 이하         | 9,406만원 + (3억원을 초과하는 금액의 38퍼센트)   |\n\n| 5억원 초과      10억원 이하        | 1억 7,406만원 + (5억원을 초과하는 금액의 42퍼센트)|\n\n| 10억원 초과        | 3억 8,406만원 + (10억원을 초과하는 금액의 45퍼센트)|\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소

In [26]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model=Constants.MODEL_CHATGPT)

In [45]:
from langchain_core.prompts import PromptTemplate

template = """[Identity]
- 당신은 최고의 한국 소득세 전문가 입니다.
- [Context]를 참고해서 사용자의 질문에 답변해주세요.

[Context]
{retrieved_docs}

Question: {query}
"""

rag_template = PromptTemplate(
    template=template,
    input_variables=['retrieved_docs', 'query'])
rag_chain = rag_template | llm # LCEL
ai_message = rag_chain.invoke({
    'retrieved_docs': retrieved_docs,
    'query': query
})

In [47]:
# prompt = f"""[Identity]
# - 당신은 최고의 한국 소득세 전문가 입니다.
# - [Context]를 참고해서 사용자의 질문에 답변해주세요.
#
# [Context]
# {retrieved_docs}
#
# Question: {query}
# """

# ai_message = llm.invoke(prompt)

In [46]:
ai_message.content

'연봉 5천만원인 직장인의 소득세를 계산하기 위해 우선 해당 연봉에 맞는 종합소득세율 구조를 확인해야 합니다. 연봉 5천만원은 다음의 구간에 해당합니다.\n\n- 1,400만원 초과 5,000만원 이하: 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)\n\n따라서, 세금을 계산해보겠습니다.\n\n1. 1,400만원에 대한 세금: 84만원\n2. 5,000만원에서 1,400만원을 뺀 금액: 5,000만원 - 1,400만원 = 3,600만원\n3. 이 금액에 15%를 곱합니다: 3,600만원 × 15% = 540만원\n\n따라서, 종합소득세는 다음과 같습니다.\n\n\\[ \n\\text{총 세액} = 84만원 + 540만원 = 624만원 \n\\]\n\n결론적으로, 연봉 5천만원인 직장인의 소득세는 **624만원**입니다.'

In [40]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

dictionary = ["사람을 나타내는 표현 -> 거주자"]

prompt = ChatPromptTemplate.from_template(f"""
    사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
    만약 변경할 필요가 없다고 판단되면, 사용자의 질문을 변경하지 않아도 됩니다.
    그런 경우에는 질문만 리턴해주세요
    사전: {dictionary}

    질문: {{query}}
""")
dictionary_chain = prompt | llm | StrOutputParser()

In [55]:
new_question = dictionary_chain.invoke({"query": query})

In [56]:
query

'연봉 5천만원인 직장인의 소득세는 얼마인가요?'

In [57]:
new_question

'변경된 질문: 연봉 5천만원인 거주자의 소득세는 얼마인가요?'

In [68]:
tax_chain = {"query": dictionary_chain, "retrieved_docs": lambda x: retrieved_docs} | rag_chain

In [69]:
ai_response = tax_chain.invoke({'retrieved_docs': retrieved_docs, "query": query})

In [70]:
ai_response

AIMessage(content='연봉 5천만원인 거주자의 소득세를 계산해보겠습니다. 2023년 소득세 세율에 따르면, 5천만원은 다음의 구간에 해당합니다:\n\n1. **5,000만원 초과 8,800만원 이하**: \n   - 세액: 624만원 + (5,000만원을 초과하는 금액의 24퍼센트)\n\n여기서 5,000만원을 초과한 금액은 5,000만원에서 5,000만원을 제외한 0원이므로, 여기서 소득세를 계산합니다.\n\n따라서 소득세는:\n- 기본 세액: 624만원\n- 초과 금액: 0원 × 24% = 0원\n\n이를 통해 최종 소득세는 624만원입니다.\n\n결론적으로, 연봉 5천만원인 거주자의 소득세는 **624만원**입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 205, 'prompt_tokens': 4122, 'total_tokens': 4327, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-0677d9a4-83cb-4d12-8973-2c001a43e522-0', usage_metadata={'input_tokens': 4122, 'output_tokens': 205, 'total_tokens': 4327, 'input_token_details': {'audio': 0, 